<a href="https://colab.research.google.com/github/mwang-cmn/Python-notebooks/blob/main/Cleaning_Bank_Data_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cleaning Bank Data in Python**

Personal loans are a lucrative revenue stream for banks.

In this project, I am working with a bank to clean and organise data collected during a recent marketing campaign. The campaign aimed to encourage customers to take out personal loans, and the bank plans to run more campaigns in the future. My role is to ensure the data meets their specified structure and formats, making it ready for use in a PostgreSQL database.

I will process a supplied CSV file, "**bank_marketing.csv**," by cleaning, reformatting, and splitting the data into three structured CSV files.
This will allow the bank to store the cleaned data efficiently and seamlessly integrate future campaign data into their system.
The three files should have the names and contents as outlined below:

**client.csv**

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

**campaign.csv**

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

**economics.csv**

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [ ]:
df['education'].value_counts()

university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

### **Data Cleaning**  

To ensure consistency and improve the quality of the dataset, I applied several data cleaning steps.  

1. **Standardising Text Data:**  
   - Replaced periods (`.`) in job and education entries with underscores (`_`) to maintain uniformity.  
   - Capitalised month names for consistency in formatting.  

2. **Handling Missing and Unknown Values:**  
   - Converted instances of `"unknown"` in the education column to `NaN`, making it easier to handle missing data during analysis.  

3. **Transforming Categorical Data:**  
   - Converted `credit_default`, `mortgage`, `previous_outcome`, and `campaign_outcome` into boolean values (`True` or `False`).  
   - Mapped `"yes"` entries to `1` and `"no"` entries to `0` for clearer representation in analyses.  
   - Converted `"success"` in `previous_outcome` to `1` and other values to `0`, making it easier to assess past campaign results.  

4. **Creating a Date Column:**  
   - Added a new `year` column, setting its value to 2022 to ensure consistency.  
   - Combined `day`, `month`, and `year` to create a proper `datetime` column (`last_contact_date`), which allows for easier time-based analysis.

In [ ]:
# Data cleaning
df['job'] = df['job'].str.replace(".", "_")
df['education'] = df['education'].str.replace(".", "_")
df['education'] = df['education'].replace("unknown",np.nan)
df['credit_default'] = df['credit_default'].map(lambda x: 1 if x == "yes" else 0).astype('bool')
df['mortgage'] = df['mortgage'].map(lambda x: 1 if x == "yes" else 0).astype('bool')
df['previous_outcome'] = df['previous_outcome'].map(lambda x: 1 if x == "success" else 0).astype('bool')
df['campaign_outcome'] = df['campaign_outcome'].map(lambda x: 1 if x == "yes" else 0).astype('bool')
df['year'] = 2022
df['month'] = df['month'].str.capitalize()
df['last_contact_date'] = pd.to_datetime(df['year'].astype(str)+"-"+df['month'].astype(str)+"-"+df['day'].astype(str), format="%Y-%b-%d")

df.head(10)

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year,last_contact_date
0,0,56,housemaid,married,basic_4y,False,False,May,13,261,1,0,False,93.994,4.857,False,2022,2022-05-13
1,1,57,services,married,high_school,False,False,May,19,149,1,0,False,93.994,4.857,False,2022,2022-05-19
2,2,37,services,married,high_school,False,True,May,23,226,1,0,False,93.994,4.857,False,2022,2022-05-23
3,3,40,admin_,married,basic_6y,False,False,May,27,151,1,0,False,93.994,4.857,False,2022,2022-05-27
4,4,56,services,married,high_school,False,False,May,3,307,1,0,False,93.994,4.857,False,2022,2022-05-03
5,5,45,services,married,basic_9y,False,False,May,5,198,1,0,False,93.994,4.857,False,2022,2022-05-05
6,6,59,admin_,married,professional_course,False,False,May,3,139,1,0,False,93.994,4.857,False,2022,2022-05-03
7,7,41,blue-collar,married,NaN,False,False,May,12,217,1,0,False,93.994,4.857,False,2022,2022-05-12
8,8,24,technician,single,professional_course,False,True,May,21,380,1,0,False,93.994,4.857,False,2022,2022-05-21
9,9,25,services,single,high_school,False,True,May,5,50,1,0,False,93.994,4.857,False,2022,2022-05-05


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   age                         41188 non-null  int64         
 2   job                         41188 non-null  object        
 3   marital                     41188 non-null  object        
 4   education                   39457 non-null  object        
 5   credit_default              41188 non-null  bool          
 6   mortgage                    41188 non-null  bool          
 7   month                       41188 non-null  object        
 8   day                         41188 non-null  int64         
 9   contact_duration            41188 non-null  int64         
 10  number_contacts             41188 non-null  int64         
 11  previous_campaign_contacts  41188 non-null  int64     

In [ ]:
# Split the Dataframe to 3 datasets
client = df[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]
campaign = df[["client_id", "contact_duration", "number_contacts", "previous_campaign_contacts", "previous_outcome", "campaign_outcome", "last_contact_date"]]
economics = df[["client_id", "cons_price_idx", "euribor_three_months"]]

In [ ]:
# Save to .csv formats
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

**Summary**

This data cleaning process ensures that the bank’s marketing data is structured, accurate and ready for efficient storage in a PostgreSQL database. By standardising formats, handling missing values and transforming key variables, I have improved data consistency and usability. This will allow the bank to seamlessly integrate future campaign data and conduct more reliable analyses, leading to better decision-making.